In [1]:
!pip install torch torchvision torchaudio torch-geometric scikit-learn numpy pandas matplotlib seaborn tqdm easydict -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.4 MB/s eta 0:00:00a 0:00:01


In [2]:
import torch, numpy as np, pandas as pd
from IPython.display import Image

In [3]:
# loaded_data = torch.load('/kaggle/input/gremi-ori/BRCA/data.pt')
# print(loaded_data.keys())

# for key in loaded_data.keys():
#     print(key)
#     print(loaded_data[key].shape)
#     print(loaded_data[key])

In [4]:
for i in range(1, 5):
    main_gremi = '/kaggle/input/gremi-ori/train_test_1.py'
    data_dir = f'/kaggle/input/gremi-gbm/TCGA_GBM_GExCNVxMETH_2000_MinMaxScaler/{i}'
    model_save_dir = f'/kaggle/working/model-gbm{i}.pth'
    patience = 100
    tr_label='tr'
    te_label='te'
    batch_size=1
    !python {main_gremi} '{data_dir}' '{model_save_dir}' '{patience}' '{tr_label}' '{te_label}' '{batch_size}'
    
    # display(Image("/kaggle/working/total_loss.png"))

 Epoch 0/2999
----------
Traceback (most recent call last):
  File "/kaggle/input/gremi-ori/train_test_1.py", line 153, in <module>
    ) = network(batch_x1, batch_x2, batch_x3, exp_adj1_device, exp_adj2_device, exp_adj3_device, targets)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File "/kaggle/input/gremi-ori/model_GREMI.py", line 81, in forward
    MMLoss = torch.mean(criterion(MMlogit, label))
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/

In [ ]:
# %cd /kaggle/input/gremi-ori/ROSMAP
# !python model_test_rosmap.py

main_gremi = '/kaggle/input/gremi-ori/train_test_0.py'
data_dir = '/kaggle/input/gremi-ori/ROSMAP/data.pt'
model_save_dir = '/kaggle/working/model-rosmap1.pth'
patience = 100
!python {main_gremi} '{data_dir}' '{model_save_dir}' '{patience}'

display(Image("/kaggle/working/total_loss.png"))

/kaggle/input/gremi-ori/train_test_0.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(data_dir)
 Epoch 0/2999
----------
0 Train Loss : 8.884401

In [ ]:
%cd /kaggle/input/gremi-ori/BRCA
!python model_test_0.py

# main_gremi = '/kaggle/input/gremi-ori/train_test_0.py'
# data_dir = '/kaggle/input/gremi-ori/BRCA/data.pt'
# model_save_dir = '/kaggle/working/model-brca1.pth'
# patience = 100
# !python {main_gremi} '{data_dir}' '{model_save_dir}' '{patience}'

# display(Image("/kaggle/working/total_loss.png"))

In [ ]:
# %cd /kaggle/input/gremi-ori/in-house data/LUAD
# !python train_test_0.py
main_gremi = '/kaggle/input/gremi-ori/train_test_0.py'
data_dir = '/kaggle/input/gremi-ori/in-house data/LUAD/data.pt'
model_save_dir = '/kaggle/working/model-luad1.pth'
patience = 100
!python {main_gremi} '{data_dir}' '{model_save_dir}' '{patience}'

display(Image("/kaggle/working/total_loss.png"))

In [ ]:
main_gremi = '/kaggle/input/gremi-ori/train_test_0.py'
data_dir = '/kaggle/input/gremi-ori/in-house data/THCA/data.pt'
model_save_dir = '/kaggle/working/model-thca1.pth'
patience = 100
!python {main_gremi} '{data_dir}' '{model_save_dir}' '{patience}'

display(Image("/kaggle/working/total_loss.png"))

In [ ]:
main_gremi = '/kaggle/input/gremi-ori/train_test_0.py'
data_dir = '/kaggle/input/gremi-ori/in-house data/UCEC/data.pt'
model_save_dir = '/kaggle/working/model-ucec1.pth'
patience = 100
!python {main_gremi} '{data_dir}' '{model_save_dir}' '{patience}'

display(Image("/kaggle/working/total_loss.png"))

In [ ]:
# %cd /kaggle/input/gremi-ori/explain
# !python mutag_sub_demo.py